# 程序说明

> 这是一个使用卷积网络在CIFAR10数据集上做分类的程序，其中使用了数据增强

> 数据集：CIFAR10

# 加载keras模块

In [2]:
from __future__ import print_function

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils

Using TensorFlow backend.


# 变量初始化

In [3]:
batch_size = 32
nb_classes = 10
nb_epoch = 20
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32

# the CIFAR10 images are RGB三通道
img_channels = 3

# 准备数据

In [10]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


# 数据归一化（压缩为0~1之间）

In [11]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# 转换类标号

In [12]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print(y_train.shape, Y_train.shape)

(50000, 1) (50000, 10)


# 建立模型

In [13]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., padding="same")`
  after removing the cwd from sys.path.
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  del sys.path[0]


## 打印模型

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_14 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        18496     
__________

# 训练与评估

## 编译模型

In [15]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(
    loss='categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy']
)

## 数据增强
数据增强使用的**ImageDataGenerator**这个函数

In [16]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # 这将做预处理和实时数据增加
    datagen = ImageDataGenerator(
        featurewise_center=False,  # 在数据集上将输入平均值设置为0
        samplewise_center=False,   # 将每个样本均值设置为0
        featurewise_std_normalization=False,  # 将输入除以数据集的std
        samplewise_std_normalization=False,   # 将每个输入除以其std
        zca_whitening=False,  # 应用ZCA白化
        rotation_range=0,     # 在一个范围下随机旋转图像(degrees, 0 to 180)
        width_shift_range=0.1,   # 水平随机移位图像（总宽度的分数）
        height_shift_range=0.1,  # 随机地垂直移动图像（总高度的分数）
        horizontal_flip=True,   # 随机翻转图像
        vertical_flip=False)    # 随机翻转图像

    # 计算特征方向归一化所需的数量
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(
        datagen.flow(
            X_train, Y_train,
            batch_size=batch_size
        ),
        samples_per_epoch=X_train.shape[0],
        nb_epoch=nb_epoch,
        validation_data=(X_test, Y_test)
    )

Using real-time data augmentation.
Instructions for updating:
Use tf.cast instead.


D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., steps_per_epoch=1562, epochs=20)`


Epoch 1/20
1562/1562 [==============================] - 180s 115ms/step - loss: 1.7997 - acc: 0.3317 - val_loss: 1.3777 - val_acc: 0.4891
Epoch 2/20
1562/1562 [==============================] - 159s 102ms/step - loss: 1.4469 - acc: 0.4766 - val_loss: 1.1925 - val_acc: 0.5709
Epoch 3/20
1562/1562 [==============================] - 150s 96ms/step - loss: 1.2896 - acc: 0.5375 - val_loss: 1.0689 - val_acc: 0.6249
Epoch 4/20
1562/1562 [==============================] - 151s 97ms/step - loss: 1.2024 - acc: 0.5713 - val_loss: 1.0826 - val_acc: 0.6138
Epoch 5/20
1562/1562 [==============================] - 150s 96ms/step - loss: 1.1292 - acc: 0.5978 - val_loss: 1.0223 - val_acc: 0.6357
Epoch 6/20
1562/1562 [==============================] - 150s 96ms/step - loss: 1.0776 - acc: 0.6181 - val_loss: 0.9573 - val_acc: 0.6665
Epoch 7/20
1562/1562 [==============================] - 149s 95ms/step - loss: 1.0412 - acc: 0.6331 - val_loss: 0.8425 - val_acc: 0.7060
Epoch 8/20
1562/1562 [=================